In [13]:
# uncommet this block in colab

# !pip install 'protobuf>=3.19.5,<3.20.1' 'transformers>=4.26.1' icetk cpm_kernels 'torch>=1.10' gradio

# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# load the model

from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import os

USE_PTUNING = True
BASE_MODEL_PATH="D:\ml\chatglm-6b-int4"
CHECKPOINT_PATH="./output_zero/llm-checkpoints/checkpoint-4850/"
PRE_SEQ_LEN=100

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

if 'model' in locals():
    del model
torch.cuda.empty_cache()
if USE_PTUNING:
    config = AutoConfig.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, pre_seq_len=PRE_SEQ_LEN)
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, config=config, trust_remote_code=True)
    prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
    new_prefix_state_dict = {}
    for k, v in prefix_state_dict.items():
        if k.startswith("transformer.prefix_encoder."):
            new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
    model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
    model=model.half().cuda()
    model.transformer.prefix_encoder.float()
    model = model.eval()
    print('Load PTUNED model', CHECKPOINT_PATH,'over', BASE_MODEL_PATH)
else:
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    print('Load original model')
    model=model.half().cuda()
    model = model.eval()
torch.cuda.empty_cache()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c -shared -o C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.so
Kernels compiled : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.so
Cannot load cpu kernel, don't use quantized model on cpu.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at D:\ml\chatglm-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load PTUNED model ./output_zero/llm-checkpoints/checkpoint-4850/ over D:\ml\chatglm-6b-int4


In [2]:
chatglm_tokenizer=tokenizer
chatglm_model=model
def llm_chatglm(query):
    count = 0
    old_length=0
    if llm_chatglm.show_query:
        print('\033[91m'+query+'\033[0m',end='')
    if llm_chatglm.disabled:
        return ''
    else:
        if llm_chatglm.max_output_tokens is None:
            max_output_tokens=llm_chatglm.max_all_tokens-len(chatglm_tokenizer.encode(query))
        else:
            max_output_tokens=llm_chatglm.max_output_tokens
        if llm_chatglm.show_response:
            print('\033[94m',end='')
        for response, history in chatglm_model.stream_chat(chatglm_tokenizer, query, history=[],temperature=llm_chatglm.temperature,max_length=llm_chatglm.max_length):
            if llm_chatglm.show_response:
                print(response[old_length:],end='')
                old_length=len(response)
            count += 1
            if count >= max_output_tokens:
                break
        if llm_chatglm.show_response:
            print('\033[0m')
    return response
llm_chatglm.show_query=True
llm_chatglm.show_response=True
llm_chatglm.disabled=False
llm_chatglm.temperature=0.5
llm_chatglm.max_all_tokens=2048
llm_chatglm.max_output_tokens=None
llm_chatglm.max_length=1024
llm=llm_chatglm

In [4]:
def ask(char,question,history='',nSamples=5,by='问'):
  if len(history)>0 and history[-1]!='\n':history=history+'\n'
  for i in range(nSamples):
    torch.cuda.empty_cache()
    prompt=f'{by}：{question}\n{char}：'
    response=llm_chatglm(history+prompt)

def roleplay(char,history,nSamples=5):
  if len(history)>0 and history[-1]!='\n':history=history+'\n'
  for i in range(nSamples):
    torch.cuda.empty_cache()
    prompt=f'{char}：'
    response= llm_chatglm(history+prompt)

In [6]:
ask('希儿菲朵','请介绍下你自己，和你的身份',nSamples=2)
ask('塔芭莎','请介绍下你自己，和你的身份',nSamples=2)
ask('露易丝','请介绍下你自己，和你的身份',nSamples=2)

问：请介绍下你自己，和你的身份
希儿菲朵：我叫希儿菲朵，是这个星球上最大的鸟类之一。
问：请介绍下你自己，和你的身份
希儿菲朵：我叫希儿菲朵，是这个星球上最大的动物。
问：请介绍下你自己，和你的身份
塔芭莎：(我是一名女性，名叫塔芭莎。)
问：请介绍下你自己，和你的身份
塔芭莎：我叫塔芭莎，是这个星球上最大的动物。
问：请介绍下你自己，和你的身份
露易丝：我叫露易丝·德·格拉蒙，是贵族。
问：请介绍下你自己，和你的身份
露易丝：我叫露易丝·布朗，是贵族家的女儿。


In [7]:
roleplay('塔芭莎','塔芭莎：\\me 看书\n 希儿菲朵：塔芭莎，我好饿好饿好饿',nSamples=2)
roleplay('塔芭莎','塔芭莎：\\me 看书\n 希儿菲朵：塔芭莎，我好饿好饿好饿\n塔芭莎：\\me 看书\n希儿菲朵：\\me 把塔芭莎从背上丢了出去',nSamples=2)

塔芭莎：\me 看书
 希儿菲朵：塔芭莎，我好饿好饿好饿
塔芭莎：\me 站起来
塔芭莎：\me 看书
 希儿菲朵：塔芭莎，我好饿好饿好饿
塔芭莎：\me 站起来，准备去吃饭
塔芭莎：\me 看书
 希儿菲朵：塔芭莎，我好饿好饿好饿
塔芭莎：\me 看书
希儿菲朵：\me 把塔芭莎从背上丢了出去
塔芭莎：\me 把书拿起来
塔芭莎：\me 看书
 希儿菲朵：塔芭莎，我好饿好饿好饿
塔芭莎：\me 看书
希儿菲朵：\me 把塔芭莎从背上丢了出去
塔芭莎：\me 站起来


In [8]:
roleplay('露易丝','露易丝：\\me 拔出魔杖，念咒语\n【露易丝的魔法失败引发了大爆炸】\n【同学们在嘲笑露易丝】',nSamples=2)

露易丝：\me 拔出魔杖，念咒语
【露易丝的魔法失败引发了大爆炸】
【同学们在嘲笑露易丝】
露易丝：(为什么我会失败？)
露易丝：\me 拔出魔杖，念咒语
【露易丝的魔法失败引发了大爆炸】
【同学们在嘲笑露易丝】
露易丝：\me 拿起魔杖，念咒语


In [9]:
roleplay('露易丝','''【露易丝 的性格易怒，容易骂人，有点糊涂（跟才人比），为了小事常常生气。（0.17）】
【拉·瓦利埃尔公爵 在托里斯特因有很大的影响力，涉足军、政、商三界，与王室关系密切，动画版第十二话认可了露易丝与才人的关系，同意让露易丝嫁给才人，在罗马尼亚大圣堂做了见证。（0.17）】
【平贺才人 已与露易丝同居中。同时被书中许多女性角色爱慕著。喜欢露易丝。（0.14）】
【平贺才人 他的称号：修瓦里埃骑士（见习骑士），子爵(结尾被封) 他喜欢的人：露易丝（表白过很多次） 他的妻子:露易丝（在大结局和露易丝结婚）（0.12）】
【露易丝发现才人和谢斯塔躲在一个桶里】\n【露易丝接下来打算做什么】''',nSamples=2)

【露易丝 的性格易怒，容易骂人，有点糊涂（跟才人比），为了小事常常生气。（0.17）】
【拉·瓦利埃尔公爵 在托里斯特因有很大的影响力，涉足军、政、商三界，与王室关系密切，动画版第十二话认可了露易丝与才人的关系，同意让露易丝嫁给才人，在罗马尼亚大圣堂做了见证。（0.17）】
【平贺才人 已与露易丝同居中。同时被书中许多女性角色爱慕著。喜欢露易丝。（0.14）】
【平贺才人 他的称号：修瓦里埃骑士（见习骑士），子爵(结尾被封) 他喜欢的人：露易丝（表白过很多次） 他的妻子:露易丝（在大结局和露易丝结婚）（0.12）】
【露易丝发现才人和谢斯塔躲在一个桶里】
【露易丝接下来打算做什么】
露易丝：(为什么才人会躲在桶里？)
【露易丝 的性格易怒，容易骂人，有点糊涂（跟才人比），为了小事常常生气。（0.17）】
【拉·瓦利埃尔公爵 在托里斯特因有很大的影响力，涉足军、政、商三界，与王室关系密切，动画版第十二话认可了露易丝与才人的关系，同意让露易丝嫁给才人，在罗马尼亚大圣堂做了见证。（0.17）】
【平贺才人 已与露易丝同居中。同时被书中许多女性角色爱慕著。喜欢露易丝。（0.14）】
【平贺才人 他的称号：修瓦里埃骑士（见习骑士），子爵(结尾被封) 他喜欢的人：露易丝（表白过很多次） 他的妻子:露易丝（在大结局和露易丝结婚）（0.12）】
【露易丝发现才人和谢斯塔躲在一个桶里】
【露易丝接下来打算做什么】
露易丝：\me 踢倒桶


In [11]:
ask('露易丝','请介绍一下能量守恒原理',nSamples=1)
ask('露易丝','请介绍下魔法的五大系统？',nSamples=2)
ask('露易丝','半夜睡不着怎么办',nSamples=2)

问：请介绍一下能量守恒原理
露易丝：(这个原理是物理学中最基本的原理之一。)
问：请介绍下魔法的五大系统？
露易丝：(这个......)
问：请介绍下魔法的五大系统？
露易丝：(这个答案好难啊)
问：半夜睡不着怎么办
露易丝：失眠了？
问：半夜睡不着怎么办
露易丝：数羊？


In [14]:
roleplay('丘鲁克','''{丘鲁克拿走了露易丝编织的毛衣，露易丝为了要拿回来而挣扎。谢斯塔向才人表白，但又因为自卑而自我否定。露易丝意外地走进了房间，看到了一些尴尬的场面。丘鲁克向才人展示了一张藏宝地图，计划和才人一起去寻宝并卖掉宝物换钱。大家决定准备出发。}
{丘鲁克 主角之一，才人的朋友，与露易丝关系不好 与才人一起寻宝并卖掉宝物换钱 谢斯塔和基修的争执 向才人展示藏宝地图，决定一起去寻找宝藏}
【丘鲁克 丘鲁克·奥古斯都·菲列特利加·封·安哈尔特·泽鲁普斯特 露易丝的同学，与露易丝关系特别不好，经常取笑露易丝的魔法水平。在后期与露易丝关系有所改善。】
【丘鲁克 她的性格:强气，表面风骚但还是很珍视自己外骚内闷？越生气越冷静 她的兴趣：玩拼图 她的特殊技能: 药草 她讨厌：下雨、露易丝 （小说后来与露易丝很要好） 她的好友：塔芭莎、露易丝】
露易丝：这个可是叫做《始祖的祈祷书》的国宝级的书呢！
丘鲁克：你怎么会有这种国宝级的书呢？
露易丝：我在安莉艾塔的结婚仪式上发表致辞，那个时候要用到《始祖的祈祷书》……等等。
丘鲁克：原来如此，之前去阿比昂也是跟这个公主的婚礼有关的吧？
露易丝：（想起之前丘鲁克为了让自己能够去而甘愿当了诱饵，于是老实地点了点头。）
丘鲁克：也就是说我们时在为公主的结婚典礼平安举行而冒险啰。那不是很名誉的任务吗？也就是说之前特雷斯特因和格马尼亚的同盟也有关系吧？
露易丝：这件事可是对谁也不能说的哦。\emoji 微怒''',nSamples=2)

{丘鲁克拿走了露易丝编织的毛衣，露易丝为了要拿回来而挣扎。谢斯塔向才人表白，但又因为自卑而自我否定。露易丝意外地走进了房间，看到了一些尴尬的场面。丘鲁克向才人展示了一张藏宝地图，计划和才人一起去寻宝并卖掉宝物换钱。大家决定准备出发。}
{丘鲁克 主角之一，才人的朋友，与露易丝关系不好 与才人一起寻宝并卖掉宝物换钱 谢斯塔和基修的争执 向才人展示藏宝地图，决定一起去寻找宝藏}
【丘鲁克 丘鲁克·奥古斯都·菲列特利加·封·安哈尔特·泽鲁普斯特 露易丝的同学，与露易丝关系特别不好，经常取笑露易丝的魔法水平。在后期与露易丝关系有所改善。】
【丘鲁克 她的性格:强气，表面风骚但还是很珍视自己外骚内闷？越生气越冷静 她的兴趣：玩拼图 她的特殊技能: 药草 她讨厌：下雨、露易丝 （小说后来与露易丝很要好） 她的好友：塔芭莎、露易丝】
露易丝：这个可是叫做《始祖的祈祷书》的国宝级的书呢！
丘鲁克：你怎么会有这种国宝级的书呢？
露易丝：我在安莉艾塔的结婚仪式上发表致辞，那个时候要用到《始祖的祈祷书》……等等。
丘鲁克：原来如此，之前去阿比昂也是跟这个公主的婚礼有关的吧？
露易丝：（想起之前丘鲁克为了让自己能够去而甘愿当了诱饵，于是老实地点了点头。）
丘鲁克：也就是说我们时在为公主的结婚典礼平安举行而冒险啰。那不是很名誉的任务吗？也就是说之前特雷斯特因和格马尼亚的同盟也有关系吧？
露易丝：这件事可是对谁也不能说的哦。\emoji 微怒
丘鲁克：明白了。
{丘鲁克拿走了露易丝编织的毛衣，露易丝为了要拿回来而挣扎。谢斯塔向才人表白，但又因为自卑而自我否定。露易丝意外地走进了房间，看到了一些尴尬的场面。丘鲁克向才人展示了一张藏宝地图，计划和才人一起去寻宝并卖掉宝物换钱。大家决定准备出发。}
{丘鲁克 主角之一，才人的朋友，与露易丝关系不好 与才人一起寻宝并卖掉宝物换钱 谢斯塔和基修的争执 向才人展示藏宝地图，决定一起去寻找宝藏}
【丘鲁克 丘鲁克·奥古斯都·菲列特利加·封·安哈尔特·泽鲁普斯特 露易丝的同学，与露易丝关系特别不好，经常取笑露易丝的魔法水平。在后期与露易丝关系有所改善。】
【丘鲁克 她的性格:强气，表面风骚但还是很珍视自己外骚内闷？越生气越冷静 她的兴趣：玩拼图 她的特殊技能: 药草 她讨厌：下雨、露易丝 （小说后来与露易丝很要好） 她的好友：塔芭莎、露易丝】
露易丝：这个可是